### This personal notebook describes notes about aws and cli
In this sample we will build in central (frankfurt) region our pred-reclaim api server.

#### Install CLI

```
Windows: https://s3.amazonaws.com/aws-cli/AWSCLI64.msi

Linux or mac requires pip to install, which requires setuptools itself. If you have pip just install the awscli)

SETUPTOOLS:
wget https://pypi.python.org/packages/source/s/setuptools/
setuptools-7.0.tar.gz
tar xvf setuptools-7.0.tar.gz
cd setuptools-7.0
python setup.py install

PIP:
wget https://bootstrap.pypa.io/get-pip.py
python get-pip.py

AWSCLI

pip install awscli

TEST:
aws -version



```
    
    

### Access and security

As we use an existing organisation for now we will not create any users and groups, maybe roles later for you api

```
Configure:

Start configure screen, if you have multiple users using aws cli 
, use different profiles.

root@master:/setuptools-7.0# aws configure (--profile <somename>)
AWS Access Key ID [None]: <secret>
AWS Secret Access Key [None]: <secret>
Default region name [None]: eu-central-1 (or eu-west-1)
Default output format [None]: table

users:
aws iam list-users
aws iam create-user --user-name <user>
aws iam create-login-profile --user-name <user> --password <pwd>
aws iam create-access-key --user <username>

groups:
aws iam create-group --group-name <grp>
aws iam add-user-to-group --user-name <user> --group-name <user>

profiles:
create this file:
{
"Version": "2012-10-17",
"Statement": [
      {
"Effect": "Allow",
"Action": "*",
"Resource": "*"
} ]
}

Grant to user (can also be group):
aws iam put-group-policy --user-name <user> \
--policy-name <all privs for your your org> \
--policy-document file:///tmp/<your file>.json

``` 

### EC2 instances

#### First some general notes

```
ec2 is container running ami images:

two types ami:
Instance store-backed instance via S# (slow)
ebs (elastic block store), shared drive, quick portable, peristant storage, also can handle 16tb instead of 10gb.  but does cost some extra

types:
memory, computer, storage or general optimised, or lastly gpu for graphical/machine learning

pricing:
on-demand , per hour, no cost when not using but higher price
reserved: cheaper on the whole, but you do pay when not using. yu can pay upfront (partly) making it cheaper.
spot: bid on capacity, no guarantee at all.

ebs features:
snapshot stored on s3 for backup and resizing , HA (replication same avail. zone), encryption optional use KMS key service

ebs volumes:
general ssd, providioned iops ssd (quick and expensive, for io intensive apps like db's and hadoop. or magnetic (cheap, backup etc)

``` 

### Install server

``` 
in our keys we will use dil-pred-reclaim-key-pair as name

public key:
aws ec2 create-key-pair --key-name dil-pred-reclaim-key-pair --query 'KeyMaterial' --output text > dil-pred-reclaim-key-pair.pem
 
chmod 400 dil-pred-reclaim-key-pair.pem

security group:
aws ec2 create-security-group --group-name sg_dil-pred-reclaim \
 --description "Security group predictive reclaim api server"

Rules:
aws ec2 authorize-security-group-ingress \
--group-name sg_dil-pred-reclaim \
--protocol tcp --port 22 --cidr 0.0.0.0/0

run (aws linux):
aws ec2 run-instances --image-id ami-b968bad6 \
 --count 1 --instance-type t2.small \
 --security-groups sg_dil-pred-reclaim \
 --key-name dil-pred-reclaim-key-pair \
 --tag-specifications 'ResourceType=instance,Tags=[{Key=Name,Value=dil-pred-reclaim}]'

test :
ssh -i dil-pred-reclaim-key-pair.pem ec2-user@<PUBLIC ip>

how to access an ebs volume you created earlier:
sudo df -h
sudo fdisk -l. (see /dev/<xx> disk not mounted and seen using df
sudo mkfs -t ext4 /dev/<xx>
sudo mkdir /somedir
sudo mount /dev/<xx> /somedir
<edit fstab to make sure it mounts after reboot>

Downside: you cant control ip, unless you create an elastic ip and attach it. Its required if you want to control ip's accessing your server
using security groups
``` 

### EBS volumes

```
use types: gp2, io1, standard (magnetic) :

aws ec2 create-volume \
--size 5 --region eu-central-1 --availability-zone eu-central-1a \
--volume-type gp2

aws ec2 attach-volume \
--volume-id <vol-xxx> \
--instance-id <i-xxx> \
--device /dev/sdg

unmount:
aws ec2 detach-volume \
--volume-id <vol-xxx>

aws ec2 delete-volume \
--volume-id <vol-xxx>

Ebs is replicated, but only in the same avail.zon.
Use snapshots to make backups to other zone.
```

### Virtual Private Cloud (VPC)

```
ITS FREE

In vpc you can create subnets with own policies and routing, seperate from the security groups. Also extra layer: Acl access control list (per subnet) for extra control. connect your own datacentre using virtual private gateway, and connect it using the datacentre gateway.

subnet: range of ip's . two types: public and private subnet (determines internet reachable or not). 

example VPC uses CIDR 10.0.0.0./16 , /16 means 65k addresses, 10.0.0.0 - 10.0.255.255 . Create subnet 10.0.1.0/24 for webserver (public) and 10.0.5.0/24 for databases (private) etc etc (/24 is 256 ip's ).

When creating default vpc: /16 (65k) ip's . az's in default region. a public and private subnet. internet gateway (router between two subnets) . some routetables, security groups and acl's so internet can be used. NOTE: ach instance by default is using the default vpc!

network acl: rules with a number (small is highest prio) allowing or denying access. FInishing with a * to block anything not in a rule.

routing table: diect traffix to from subnet. main route table is default. and it allows traffic inside the vpc.

vpc endpoint: connect vpc to other services (virtual devices, managed by aws)

internet gateway: provide internet connectivity. just add route in routetable of subnet to it and its ready. It also nat's private ip's to public or elastic ip's . the instances themselves dont know the public ip's .

NAT instance: Without internet gatway you can use this nat in public subnet, and route instance in private subnet to this public subnet, which routes to internet (via internet gateway)

DNS Domain name servers. Domain name
```

### VPC creation

### Scenarios

```
scenarios:
1 1 single public subnet . Most simple, all connected to internet
2 2 subnet, 1 publiv 1 private (NAT). Most common. web servers in public, db in private. WIth nat in public for outgoing traffic from private subnet
3 like nr 2 but also vpn access (virtual private gateway). Used to extend on premise datacentre to cloud !!! 
4 only private subnet with vpc . Only internal connections can be made  np internet gateway 

Choice: 3. As we need on premise access, but also expect other clients like klm to connect. But also we need outgoing access as we get lots of internet data (think IOT, flighttracker etc). VPN costs 5 cents per (partial) hour, also for NAT server Extra costs per GB. 
```


### Requirements

```
requirements:

VPC SDH-EU-CENTRAL-PROD-1 192.168.0.0/16  
questions:
- 65k ip's to much? We need no-sql db's, sql-db's , streaming clusters , bigdata cluster. 
- Use central/frankfurt or ireland? Is USA allowed, for example for second Avail. zone?

SDH-EU-CENTRAL-PROD-WEB 192.168.1.0/24 (255 ip's with internet access)
, in zone EU-CENTRAL-1a

SHD-EU-CENTRAL-PROD-DB 192.168.5.0/24 (255 ip's isolated from internet),
in avail. zoine EU-CENTRAL-1b

NAT server in public subnet to have outgoing internet access via internet gateway

How to handle access to S3 etc. For example to ingest flighttracker data to S3 a api in public subnet needs it. So Spark in the private subnet can process it. 
questions:
- full access or custom?

TODO Avail. zones. Boths are in EU-CENTRAL-1a . We need duplicate environment in EU-central-1b? Not sure how this is setup best

Tenancy : dedicated (extra costs but safer)
```

### Manual vpc creation

```
VPC wizard (from vpc dashboard)
- Click option 3 (CANT BE DONE WITHOUT VPN GATEWAY INFO)
So we pick 2. For the nat server we use micro t2, with dil-keypair.
We should create seperate keys first.

Questions:
- Failed with selecting dedicated hardware, but ok when default. Problem?
- Ec2 instance created, but does not seem to be used as NAT server. As 
NAT section shows no NAT server. Meaning private subnet instance cant use internet as NAT is missing. Delete ec2 instance and create NAT instance manually?

```
    

### Aws cli vpc

```
Create public and private subnet based on output:
use --dry-run to test first.

aws ec2 create-vpc --cidr-block 192.168.0.0/16 \
--instance-tenancy dedicated 
aws ec2 create-tags --resources <vpc*> --tags Key=Name,Value=SDH-EU-CENTRAL-PROD-1
aws ec2 create-subnet --vpc-id <vpc-*> --cidr-block 192.168.1.0/24
aws ec2 create-tags --resources <subnet*> --tags Key=Name,Value=SDH-EU-CENTRAL-PROD-WEB
aws ec2 create-subnet --vpc-id <vpc-*> --cidr-block 192.168.5.0/24
aws ec2 create-tags --resources <subnet*> --tags Key=Name,Value=SDH-EU-CENTRAL-PROD-DB
aws ec2 create-internet-gateway
aws ec2 attach-internet-gateway --internet-gateway-id <igw*> --vpc-id <vpc-*> 
aws ec2 create-route --route-table-id <rtb*> --destination-cidr-block 0.0.0.0/0 --gateway-id <igw*>
aws ec2 associate-route-table --route-table-id <rtb*> --subnet-id <subnet-public*>

```

Test:
aws ec2 run-instances --image-id ami-9abea4fb --count 1 --instance-type t2.micro --key-name <key*> --security-group-ids <sg-*> --subnet-id <subnet*> --associate-public-ip-address